In [ ]:
import numpy as np
import pandas as pd
from scipy import stats

#importing the models
import Kmeans
import ALS
import NN

In [ ]:
def median_vote(predictions):
    #computing the median
    median=np.median(predictions,axis=0)
    #making sure the result is an int (not the case if the amount of predictions is even)
    bounded_median=np.floor(median)
    return bounded_median

In [ ]:
def mode_vote(predictions):
    #computing the mode
    return stats.mode(predictions,axis=0)[0]

In [ ]:
def vote(voting_f):
    #useful constants
    submission_path='submission.csv'
    training_path = "data/data_train.csv"
    format_path = "data/sampleSubmission.csv"
    #Initializing 'predictions' and 'labels' using a first prediction
    predictions=ALS.main(training_path, format_path).to_numpy()
    labels=predictions[:,0]
    predictions=predictions[:,1]
    #computing multiple predictions of the kmeans algorithm
    for k in [3,6,7]:
        predictions=np.vstack((predictions,Kmeans.main(training_path, format_path, k).to_numpy()[:,1]))
    
    predictions=np.vstack((predictions, NN.main(training_path, format_path).to_numpy()[:,1]))
    
    #finding the best prediction though the voting function
    pred=pd.DataFrame(np.vstack((labels,voting_f(predictions))).T)
    pred=pred.rename(columns={0: 'Id', 1:'Prediction'})
    #exporting the final prediction using the submission path
    pred.to_csv(submission_path)

In [ ]:
vote(median_vote)

In [ ]:
#useful constants
submission_path='submission.csv'
training_path = "data/data_train.csv"
format_path = "data/sampleSubmission.csv"
#Initializing 'predictions' and 'labels' using a first prediction
predictions_als=ALS.main(training_path, format_path)
predictions_als.set_index("Id", inplace=True)


In [ ]:
#computing multiple predictions of the kmeans algorithm
k=7
predictions_kmeans = Kmeans.main(training_path, format_path, k)
predictions_kmeans.set_index("Id", inplace=True)

In [ ]:
predictions_nn = NN.main(training_path, format_path)
predictions_nn.set_index("Id", inplace=True)

In [ ]:
concat = pd.merge(pd.merge(predictions_kmeans, predictions_als, left_index=True, right_index=True), predictions_nn, left_index=True, right_index=True)
concat.columns = ["Pred1", "Pred2", "Pred3"]
concat["Prediction"] = concat.median(axis=1)
concat.drop(["Pred1", "Pred2", "Pred3"], axis=1, inplace=True)

In [ ]:
predictions_kmeans.Prediction.value_counts()

In [ ]:
concat.to_csv(submission_path)